# Генерація Зображень за Допомогою Stable Diffusion з Використанням Euler A та DDIM

Цей Notebook демонструє, як використовувати різні методи семплінгу (`EulerA` та `DDIM`) для генерації зображень за допомогою моделі Stable Diffusion. Ви зможете налаштовувати параметри генерації, такі як кількість кроків семплінгу, насіння (seed), масштаб керування (`cfg_scale`) та інші.

In [ ]:
# Встановлення необхідних бібліотек
!pip install --upgrade diffusers transformers torch torchvision matplotlib safetensors

## Імпорт необхідних модулів

In [ ]:
from diffusers import StableDiffusionPipeline, EulerAncestralDiscreteScheduler, DDIMScheduler
from transformers import CLIPTextModel, CLIPTokenizer
import torch
from PIL import Image
import matplotlib.pyplot as plt
import random

## Перевірка наявності GPU

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Використовується пристрій: {device}")

## Завантаження токенізатора та текстового енкодера

In [ ]:
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32").to(device)

## Завантаження пайплайна Stable Diffusion з вибором методу семплінгу

In [ ]:
def load_pipeline(model_id, scheduler_type="EulerA"):
    if scheduler_type == "EulerA":
        scheduler = EulerAncestralDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
    elif scheduler_type == "DDIM":
        scheduler = DDIMScheduler.from_pretrained(model_id, subfolder="scheduler")
    else:
        raise ValueError("Невідомий тип семплінгу")
    
    pipe = StableDiffusionPipeline.from_pretrained(
        model_id,
        scheduler=scheduler,
        torch_dtype=torch.float16 if device.type == "cuda" else torch.float32
    )
    pipe = pipe.to(device)
    return pipe

## Завантаження пайплайна

In [ ]:
model_id = "CompVis/stable-diffusion-v1-4"  # Замість цього використовуйте ваш ідентифікатор моделі
scheduler_type = "EulerA"  # Можливі значення: "EulerA", "DDIM"
pipe = load_pipeline(model_id, scheduler_type)
print("Пайплайн успішно завантажено та перенесено на пристрій.")

## Функція для генерації зображення

In [ ]:
def generate_image(
    prompt,
    sampler="EulerA",
    sampling_steps=50,
    seed=None,
    cfg_scale=7.5,
    image_size=(512, 512),
    hires_fix=False,
    hires_scale=2.0,
    hires_steps=20
):
    """
    Генерує зображення на основі текстового опису з можливістю налаштування параметрів.
    
    Parameters:
    - prompt (str): Текстовий опис зображення.
    - sampler (str): Метод семплінгу ("EulerA", "DDIM").
    - sampling_steps (int): Кількість кроків семплінгу.
    - seed (int): Насіння для відтворюваності.
    - cfg_scale (float): Контролює вплив текстового опису на зображення.
    - image_size (tuple): Розмір зображення (ширина, висота).
    - hires_fix (bool): Використання високоякісного виправлення.
    - hires_scale (float): Масштаб для високоякісного виправлення.
    - hires_steps (int): Кількість кроків для високоякісного виправлення.
    
    Returns:
    - image (PIL.Image.Image): Згенероване зображення.
    """
    # Встановлення насіння для відтворюваності
    if seed is not None:
        torch.manual_seed(seed)
        random.seed(seed)
    
    # Вибір методу семплінгу
    if sampler == "EulerA":
        scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)
    elif sampler == "DDIM":
        scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
    else:
        raise ValueError("Невідомий метод семплінгу")
    
    pipe.scheduler = scheduler
    
    # Токенізація промпту з обмеженням довжини
    inputs = tokenizer(
        prompt, 
        return_tensors="pt", 
        truncation=True, 
        max_length=77
    ).to(device)
    
    # Генерація текстових ембеддінгів
    with torch.no_grad():
        text_embeddings = text_encoder(**inputs).last_hidden_state
    
    # Генерація зображення
    output = pipe(
        prompt=prompt,
        num_inference_steps=sampling_steps,
        guidance_scale=cfg_scale,
        height=image_size[1],
        width=image_size[0],
        generator=torch.Generator(device).manual_seed(seed) if seed else None
    )
    
    image = output.images[0]
    
    # Високоякісне виправлення зображення
    if hires_fix:
        # Завантаження моделі високої роздільної здатності (потрібна відповідна модель)
        hires_model_id = "NKMD/Superscale-8x"
        hires_pipe = load_pipeline(hires_model_id, scheduler_type)
        
        # Налаштування масштабування та кроків творчості
        hires_pipe.scheduler.set_timesteps(hires_steps)
        
        # Генерація високоякісного зображення
        hires_output = hires_pipe(
            prompt=prompt,
            num_inference_steps=hires_steps,
            guidance_scale=hires_scale,
            height=int(image_size[1] * hires_scale),
            width=int(image_size[0] * hires_scale),
            generator=torch.Generator(device).manual_seed(seed) if seed else None
        )
        
        image = hires_output.images[0]
    
    return image

## Приклад використання функції для генерації зображення

In [ ]:
prompt = "A futuristic cityscape at sunset with flying cars and towering skyscrapers, ultra-realistic"

# Налаштування параметрів
sampler = "EulerA"          # Метод семплінгу: "EulerA" або "DDIM"
sampling_steps = 100        # Кількість кроків семплінгу
seed = 42                   # Насіння для відтворюваності
cfg_scale = 7.5             # Контроль впливу тексту
image_size = (768, 768)     # Розмір зображення (ширина, висота)
hires_fix = True            # Використання високоякісного виправлення
hires_scale = 2.0           # Масштаб для високоякісного виправлення
hires_steps = 30            # Кількість кроків для високоякісного виправлення

# Генерація зображення
generated_image = generate_image(
    prompt=prompt,
    sampler=sampler,
    sampling_steps=sampling_steps,
    seed=seed,
    cfg_scale=cfg_scale,
    image_size=image_size,
    hires_fix=hires_fix,
    hires_scale=hires_scale,
    hires_steps=hires_steps
)

# Відображення згенерованого зображення
plt.figure(figsize=(10, 10))
plt.imshow(generated_image)
plt.axis('off')
plt.show()

## Пояснення коду

### 1. Встановлення та Імпорт бібліотек
- Використовуємо `pip` для встановлення необхідних бібліотек.
- Імпортуємо модулі, необхідні для роботи з моделями та обробки зображень.

### 2. Перевірка наявності GPU
- Визначаємо, чи доступний GPU для прискорення обчислень.

### 3. Завантаження токенізатора та текстового енкодера
- Використовуємо `CLIPTokenizer` та `CLIPTextModel` для обробки текстових запитів.

### 4. Функція `load_pipeline`
- Дозволяє завантажувати пайплайн з вибором методу семплінгу (`EulerA` або `DDIM`).
- Переносить модель на доступний пристрій (GPU або CPU).

### 5. Функція `generate_image`
- Приймає різні параметри для налаштування генерації зображення.
- Встановлює насіння для відтворюваності результатів.
- Вибирає метод семплінгу та налаштовує пайплайн.
- Токенізує текстовий опис з обмеженням на максимальну довжину.
- Генерує зображення з заданими параметрами.
- Якщо увімкнено `hires_fix`, виконує високоякісне виправлення зображення.

### 6. Приклад Використання
- Визначаємо текстовий опис (`prompt`) та налаштовуємо параметри генерації.
- Викликаємо функцію `generate_image` з заданими параметрами.
- Відображаємо згенероване зображення за допомогою `matplotlib`.